##### ML FLOW ASSIGNMENT

In [132]:
from sklearn import datasets 
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import mlflow
import mlflow.sklearn


In [131]:
iris = datasets.load_iris()
X, y = iris.data, iris.target
x_train, x_test, y_train, y_test = train_test_split(X,y)

## MLFlow 
- Log param 
- Log metric 
- Serialize trained object for MLflow Models  

### Model 1 - Training SVM with different param

In [103]:
clf = svm.SVC()
param_grid = {
    'kernel': ['linear', 'rbf'], 
    'gamma' : [0.08, 0.05]
}
grid = GridSearchCV(clf, param_grid=param_grid,cv=3)
grid.fit(x_train,y_train)

In [106]:
with mlflow.start_run():
    size = len(grid.cv_results_['params'])
    for i in range(0, size):
        print('kernel', grid.cv_results_['params'][i]['kernel'])
        print('gamma', grid.cv_results_['params'][i]['gamma'])
        mlflow.log_param('kernel', grid.cv_results_['params'][i]['kernel'])
        mlflow.log_param('gamma', grid.cv_results_['params'][i]['gamma'])
        print grid.cv_results_['mean_train_score'][i]
        mlflow.log_metric('accuracy', grid.cv_results_['mean_train_score'][i])
    clf = grid.best_estimator_
    mlflow.sklearn.log_model(clf, "SVM_model_1")
    y_predited = clf.predict(x_test)
    accuracy_score(y_test, y_predited)
    

('kernel', 'linear')
('gamma', 0.08)
1.0
('kernel', 'rbf')
('gamma', 0.08)
0.9732100521574206
('kernel', 'linear')
('gamma', 0.05)
1.0
('kernel', 'rbf')
('gamma', 0.05)
0.9732100521574206


### Model 2 - Training KNN with different param

In [133]:
knn_clf = KNeighborsClassifier()
param_grid = {
    'n_neighbors': [5, 10, 15], 
    'weights' : ['uniform', 'distance']
}
grid = GridSearchCV(knn_clf, param_grid=param_grid,cv=3)
grid.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [5, 10, 15], 'weights': ['uniform', 'distance']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [119]:
with mlflow.start_run():
    size = len(grid.cv_results_['params'])
    for i in range(0, size):
        print('n_neighbors', grid.cv_results_['params'][i]['n_neighbors'])
        print('weights', grid.cv_results_['params'][i]['weights'])
        mlflow.log_param('n_neighbors', grid.cv_results_['params'][i]['n_neighbors'])
        mlflow.log_param('weights', grid.cv_results_['params'][i]['weights'])
        print grid.cv_results_['mean_train_score'][i]
        mlflow.log_metric('accuracy', grid.cv_results_['mean_train_score'][i])
    clf = grid.best_estimator_
    mlflow.sklearn.log_model(clf, "knn_model_1")
    y_predited = clf.predict(x_test)
    accuracy_score(y_test, y_predited)

('n_neighbors', 5)
('weights', 'uniform')
0.9777145566619252
('n_neighbors', 5)
('weights', 'distance')
1.0
('n_neighbors', 10)
('weights', 'uniform')
0.9777145566619252
('n_neighbors', 10)
('weights', 'distance')
1.0
('n_neighbors', 15)
('weights', 'uniform')
0.96882408724514
('n_neighbors', 15)
('weights', 'distance')
1.0


# Model deployment / Servering 
### Run/ Start serialized MLmodels 
- Example :  mlflow sklearn serve /Users/mlruns/0/8fb04113eddb44488c2ce9c35beaf0ee/artifacts/SVM_model_1/ -p 1234
- Example :  mlflow sklearn serve /Users/mlruns/0/8fb04113eddb44488c2ce9c35beaf0ee/artifacts/KKN_model_1/ -p 1235



## Access models running in remote  server: 
-  SVM running in port 1234 
-  KNN running in port 1235 

In [126]:
import requests
test_data = [[7.1, 3. , 5.9, 2.1], [4.6, 3.1, 1.5, 0.2],[6.4, 2.9, 4.3, 1.3]]

In [138]:
# model 1
# SVM model  
r = requests.post('http://127.0.0.1:1234/invocations', json=test_data)
print r.json()

{u'predictions': [2, 0, 1]}


In [128]:
# model 2
# KKN model 
r = requests.post('http://127.0.0.1:1235/invocations', json=test_data)
print r.json()

{u'predictions': [2, 0, 1]}


### TensorFlow

In [135]:
import tensorflow as tf

hello = tf.constant('Hello, TensorFlow!')
sess = tf.Session()
print(sess.run(hello))

Hello, TensorFlow!
